In [98]:
import requests
import pandas as pd
from datetime import datetime, timedelta

url = 'https://api.stlouisfed.org/fred/series/observations'
api_key = '736152d5437ccdf966874039f99fcf78' 

# Read quarter end dates and quarter IDs from CSV file
quarter_dates_df = pd.read_csv('quarter_end_dates.csv', dtype={'quarter_end_date': str, 'quarter_id': str})

quarter_dates = quarter_dates_df[["quarter_id", "quarter_end_date"]].values

series_id_list = ['DGS10', 'FEDFUNDS']

result_data = {}  # Dictionary to store the results

for quarter_id, date in quarter_dates:
    quarter_end_date = datetime.strptime(date, '%Y%m%d')
    start_date = quarter_end_date.strftime('%Y-%m-%d')
    end_date = start_date

    for series_id in series_id_list:
        params = {
            'series_id': series_id,
            'api_key': api_key,
            'observation_start': start_date,
            'observation_end': end_date,
            'file_type': 'json'
        }

        while True:
            print("Fetching data for date:", end_date)
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                # Process the data as needed
                if data['observations']:
                    value = data['observations'][0]['value']
                    print("Quarter ID:", quarter_id)
                    print("Series ID:", series_id)
                    print("Value:", value)

                    # Check if the value is numeric
                    if value == ".":
                        # Go back one day
                        quarter_end_date -= timedelta(days=1)
                        start_date = quarter_end_date.strftime('%Y-%m-%d')
                        end_date = start_date
                        params['observation_start'] = start_date
                        params['observation_end'] = end_date
                        continue

                    # Store the value with series ID as the column name
                    if series_id in result_data:
                        result_data[series_id][quarter_id] = value
                    else:
                        result_data[series_id] = {quarter_id: value}

                    break
                else:
                    print("Data not found for date:", end_date)
            else:
                print('Error occurred. Status code:', response.status_code)

            # Go back one day
            quarter_end_date -= timedelta(days=1)
            start_date = quarter_end_date.strftime('%Y-%m-%d')
            end_date = start_date
            params['observation_start'] = start_date
            params['observation_end'] = end_date

    print()  # Add an empty line between iterations

# Create a DataFrame from the result data
result_df = pd.DataFrame(result_data)

# Save the DataFrame as fred_data.csv
result_df.to_csv('fred_data.csv', index_label='quarter_id')


Fetching data for date: 2017-03-31
Quarter ID: 2017-01
Series ID: DGS10
Value: 2.40
Fetching data for date: 2017-03-31
Quarter ID: 2017-01
Series ID: FEDFUNDS
Value: 0.79

Fetching data for date: 2017-06-30
Quarter ID: 2017-02
Series ID: DGS10
Value: 2.31
Fetching data for date: 2017-06-30
Quarter ID: 2017-02
Series ID: FEDFUNDS
Value: 1.04

Fetching data for date: 2017-09-30
Data not found for date: 2017-09-30
Fetching data for date: 2017-09-29
Quarter ID: 2017-03
Series ID: DGS10
Value: 2.33
Fetching data for date: 2017-09-29
Quarter ID: 2017-03
Series ID: FEDFUNDS
Value: 1.15

Fetching data for date: 2017-12-31
Data not found for date: 2017-12-31
Fetching data for date: 2017-12-30
Data not found for date: 2017-12-30
Fetching data for date: 2017-12-29
Quarter ID: 2017-04
Series ID: DGS10
Value: 2.40
Fetching data for date: 2017-12-29
Quarter ID: 2017-04
Series ID: FEDFUNDS
Value: 1.30

Fetching data for date: 2018-03-31
Data not found for date: 2018-03-31
Fetching data for date: 2018